In [ ]:
# dont_test
# default_exp http
%load_ext nb_black

<IPython.core.display.Javascript object>

# Async File Response

Serve a file asynchronously using [aiofiles](https://github.com/Tinche/aiofiles).

In [ ]:
# export

import time
import aiofiles


class AsyncFileResponse:
    def __init__(self, path, chunk_size=4096):
        self.path = path
        self.chunk_size = chunk_size
        self.status_code = 200
        self.raw_headers = {}

    async def stream(self, send):
        started_serving = time.perf_counter()
        await send(
            {
                "type": "http.response.start",
                "status": self.status_code,
                "headers": self.raw_headers,
            }
        )
        async with aiofiles.open(self.path, mode="rb") as file:  # type: ignore
            more_body = True
            while more_body:
                chunk = await file.read(self.chunk_size)
                more_body = len(chunk) == self.chunk_size
                await send(
                    {
                        "type": "http.response.body",
                        "body": chunk,
                        "more_body": more_body,
                    }
                )
        self.elapsed = time.perf_counter() - started_serving

<IPython.core.display.Javascript object>

## Tests

In [ ]:
import os

from pathlib import Path

path = Path("testdata.bin")

with path.open("wb") as f:
    f.write(os.urandom(10000))

with path.open("rb") as f:
    content = f.read()

chunk_datas = []


async def send(chunk_data):
    chunk_datas.append(chunk_data)


response = AsyncFileResponse(path)
await response.stream(send)
received_content = b"".join([cd.get("body", b"") for cd in chunk_datas])

assert content == received_content

<IPython.core.display.Javascript object>